In [1]:
import os
import gc
import sys
import json
import glob
import random
from datetime import datetime
import time
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import itertools
from tqdm import tqdm
import imutils

from ax import optimize
from sklearn.model_selection import RepeatedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchsummary

from torchvision import models, transforms
import torchvision

from sklearn.metrics import precision_score, recall_score, average_precision_score, f1_score

In [2]:
IMAGE_DIR = Path('/home/ubuntu/efs/kaggle/imaterialist/test')
DATA_DIR = Path('/home/ubuntu/efs/kaggle/imaterialist/')
ROOT_DIR = Path('/home/ubuntu/efs/kaggle/imaterialist/maskrcnn/logs')

# For demonstration purpose, the classification ignores attributes (only categories),
# and the image size is set to 512, which is the same as the size of submission masks
NUM_CATS = 46
IMAGE_SIZE = 512

# Model

In [3]:
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
        base_model = models.resnet101(pretrained=True)
        
        self.base = nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Sequential(
                        nn.Linear(2048,1024),
                        nn.ReLU(),
                        nn.Linear(1024,512),
                        nn.ReLU(),
                        nn.Linear(512,92),
                        nn.Sigmoid())

    def forward(self, images, class_id):
        features = self.base(images)
        #class_id_repeat = class_id.repeat(12,1,1,1)
        #class_id_repeat = class_id.view(-1,1, 1,1).repeat(1,12,1,1)
        ##print(features.shape, class_id.shape, class_id_repeat.shape)
        #x = torch.cat((features, class_id_repeat), dim=1)
        #x = x.view((-1,2060))

        y = self.fc(features.view((-1,2048)))
        return y
    
model = Resnet().to("cuda")
model = nn.DataParallel(model)

#model_path = "/home/ubuntu/efs/kaggle/imaterialist/checkpoints/attribs/20190604-1603/model_step_11653.pth"
model_path = "/home/ubuntu/efs/kaggle/imaterialist/checkpoints/attribs/20190605-1548/model_step_12864.pth"
model.load_state_dict(torch.load(model_path))
model.eval()


DataParallel(
  (module): Resnet(
    (base): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)
          (downsampl

# Data

In [4]:
class AttribsDataset(Dataset):
    def __init__(self, df):
        self.n_attributes = 92
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        row = self.df.iloc[idx]
        if row.EncodedPixels=="1 0":
            return idx, np.zeros((3,512,512),dtype=np.float32), np.float32(100)
        image_path = os.path.join(IMAGE_DIR, row.ImageId)
        image = cv2.imread(image_path)

        # mask out
        original_h, original_w, _ = image.shape
        h, w = 512, 512
        mask = np.zeros((h*w), dtype=np.uint8)
        
        encodedPixels = list(map(int,row.EncodedPixels.split(' ')))
        for startPos, runLen in zip(encodedPixels[0::2], encodedPixels[1::2]):
            mask[startPos+1:startPos+1+runLen] = 1
        mask = np.transpose(np.reshape(mask,(w,h)))
        mask = cv2.resize(mask, (original_w, original_h))

        # crop
        bx,by,bw,bh = cv2.boundingRect(mask)
        maskedImage = image* np.repeat(np.expand_dims(mask,-1), 3, axis=2)
        cropped = maskedImage[by:by+bh, bx:bx+bw,:]

        # resize largest dim to 512
        resized_cropped = self.resize_apsect_ratio(cropped, 512)
        class_id = row.ClassId
        
        resized_cropped = np.float32(resized_cropped.transpose((2, 0, 1)))/127.5 - 1
        
        return idx, resized_cropped, np.float32(class_id)
    
    def resize_apsect_ratio(self, img, dim=512):
        img_h, img_w, _ = img.shape
        if img_h>img_w:
            resized = imutils.resize(img, height=dim)
            total_pad = dim-resized.shape[1]
            l_pad = total_pad//2
            r_pad = total_pad -l_pad
            resized = cv2.copyMakeBorder(resized, 0, 0, l_pad, r_pad, cv2.BORDER_CONSTANT, None, 0.0)
        else:
            resized = imutils.resize(img, width=dim)
            total_pad = dim-resized.shape[0]
            top_pad = total_pad//2
            bottom_pad = total_pad -top_pad
            resized = cv2.copyMakeBorder(resized,top_pad, bottom_pad,  0, 0, cv2.BORDER_CONSTANT, None, 0.0)
        assert resized.shape[:2]==(dim, dim)
        return resized
    
df = pd.read_csv("input.csv")
df["ClassId"]=df["ClassId"].astype(str)
dataset = AttribsDataset(df)



# Evaluate

In [5]:
batch_size = 8
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=8)

In [6]:
threshold = 0.42658067635687696
count = 0
for idxs, images, class_ids in tqdm(dataloader):
    pred_labels = model(images.to("cuda"),
                        class_ids.to("cuda"))
    for idx, pred, class_id in zip(idxs, pred_labels, class_ids):
        if class_id<=12:
            pred = pred.cpu().detach().numpy().astype(np.float16)
            attrib_classes = np.where(pred>=threshold)[0]
            if len(attrib_classes)>0:
                attribs_idx = list(map(str,attrib_classes))
                attribs = "_".join(attribs_idx)
                labels = str(int(class_id))+'_'+attribs
                i = idx.cpu().detach().numpy().astype(np.int)
                df.at[int(i), "ClassId"] = labels
                count+=1
                if count>0:
                    break

 39%|███▉      | 847/2155 [06:03<04:25,  4.93it/s]

KeyboardInterrupt: 

In [ ]:
print(df.head())
df.to_csv("submission.csv", index=False)

In [18]:
message = "%s attribs threshold %.4f"%('/'.join(model_path.split('/')[-2:]), threshold)

In [20]:
!kaggle competitions  submit imaterialist-fashion-2019-FGVC6 -f submission.csv -m message

100%|██████████████████████████████████████| 16.0M/16.0M [00:07<00:00, 2.23MB/s]
Successfully submitted to iMaterialist (Fashion) 2019 at FGVC6 

In [11]:
!ls

attributes.ipynb	    input.csv		      __pycache__
attributes.py		    inspect_attributes.ipynb  runs
coco.py			    inspect_data.ipynb	      submission.csv
inference_attributes.ipynb  inspect_model.ipynb
inference.ipynb		    inspect_weights.ipynb


In [21]:
df.head(100)

,ImageId,EncodedPixels,ClassId
0,003d41dd20f271d27219fe7ee6de727d.jpg,87453 6 87957 20 88464 29 88972 36 89479 45 89...,8_20_42_53_61_85_91_20_42_60_91_20_41_60_62_0_...
1,003d41dd20f271d27219fe7ee6de727d.jpg,171208 9 171254 8 171705 8 171717 17 171763 19...,31
2,003d41dd20f271d27219fe7ee6de727d.jpg,72562 12 73069 20 73575 26 74079 34 74578 47 7...,31
3,003d41dd20f271d27219fe7ee6de727d.jpg,137721 5 138231 1 138233 6 138741 10 139242 7 ...,21
4,003d41dd20f271d27219fe7ee6de727d.jpg,104104 1 104615 3 105127 3 105638 5 106150 5 1...,28
5,003d41dd20f271d27219fe7ee6de727d.jpg,107180 2 107693 2 108206 2 108719 2 109231 2 1...,33
6,0046f98599f05fd7233973e430d6d04d.jpg,129665 2 129669 1 130174 13 130188 2 130686 9 ...,33
7,0046f98599f05fd7233973e430d6d04d.jpg,84891 4 85392 26 85901 31 86411 34 86921 38 87...,10_20_42_53_61_85_91_20_42_60_91_20_41_60_62_0...
8,0046f98599f05fd7233973e430d6d04d.jpg,211820 6 217485 2 217995 6 218506 9 219018 10 ...,24
9,004e9e21cd1aca568a8ffc77a54638ce.jpg,66897 45 67395 62 67901 70 68410 74 68919 78 6...,8_0_20_41_14_20_34_61_19_60_61_91_20_61_20_41_...
